# Commodity Dashboard

## Importing Data, Libraries and Scripts

Data is stored into commodity_lab/db/commodities.db 

In [8]:
import sys
print(sys.executable)

c:\Users\dgalassi\commodity_lab\env_commodity_lab\Scripts\python.exe


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as ply
import seaborn as sns
import sys

In [ ]:
#update the database with the most recent data
sys.path.append('../../scripts')
from update_commodities_data import update_commodities

#tickers i want to update
tickers = {
    "Gold": "GC=F",
    "Silver": "SI=F",
    "Crude Oil": "CL=F",
    "Natural Gas": "NG=F",
    "Copper": "HG=F",
    "Corn": "ZC=F",
    "Wheat": "ZW=F"
}
update_commodities(tickers)

In [15]:
#connect to commodities db 
import sqlite3 as sql

#enstablish a connection
conn = sql.connect("../../db/commodities.db")
df = pd.read_sql('SELECT * FROM prices',conn)
conn.close()

## Analyze the Dataset

1. Price Trends & Returns

Line charts of price history for key commodities

% change over multiple time horizons (1 day, 5 days, 1 month, 1 year)

Quick summary stats (last price, daily return, volatility)

2. Volatility & Risk Metrics

Rolling volatility (e.g., 30-day std dev)

Maybe Value at Risk (VaR) or drawdowns for the bigger picture risk view

3. Correlations

Heatmap or matrix showing correlation between commodities over the past month or quarter — helps PMs see diversification or contagion

4. Volume & Liquidity

Volume trends, so they know where the market interest is

5. Alerts or Significant Moves

Highlight big moves or breakouts (e.g., +3% daily move)

Could be a simple “flag” or conditional formatting

6. Comparative Analysis

Commodities side-by-side to see which are outperforming

### Price Trends and Returns

In [28]:
df_prices = df[['date','name','close']]
df_prices['date'] = pd.to_datetime(df.date)
df_prices.head()

C:\Users\dgalassi\AppData\Local\Temp\ipykernel_9952\911365896.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,name,close
0,2025-01-07,Gold,2656.699951
1,2025-01-08,Gold,2664.500000
2,2025-01-09,Gold,2683.800049
3,2025-01-10,Gold,2708.500000
4,2025-01-13,Gold,2673.500000


In [29]:
df_wide = df_prices.pivot(index='date',columns='name',values = 'close')
print(df_wide.shape)
df_wide.head()

(252, 7)


name,Copper,Corn,Crude Oil,Gold,Natural Gas,Silver,Wheat
date,,,,,,,
2024-07-08,4.6220,395.75,82.330002,2355.199951,2.366,30.618000,553.00
2024-07-09,4.5780,400.25,81.410004,2360.100098,2.344,30.771999,554.25
2024-07-10,4.6075,403.25,82.099998,2372.199951,2.329,30.726999,543.50
2024-07-11,4.5085,406.75,82.620003,2415.000000,2.268,31.393999,554.25
2024-07-12,4.5985,400.00,82.209999,2414.000000,2.329,30.886000,538.00


In [ ]:
df_wid

date
2024-07-08         NaN
2024-07-09         NaN
2024-07-10         NaN
2024-07-11         NaN
2024-07-12         NaN
                ...   
2025-07-01    0.037189
2025-07-02    0.048036
2025-07-03    0.006219
2025-07-04   -0.009865
2025-07-07   -0.003976
Name: Copper, Length: 252, dtype: float64

In [ ]:
import plotly.express as px
